<a href="https://colab.research.google.com/github/degartHub/nocountry-h12-25-equipo27-datascience/blob/main/H12_25_L_Equipo_27_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook para el proyecto de predicción de atrasos de vuelos - HACKATHON ONE

## Data Engineer (DE)

Sección para las tareas de Data Engineer.

Encargado: Ismael Cerda

### Selección y Limpieza de Datos

Base de datos obtenida de: https://www.kaggle.com/datasets/jimschacko/airlines-dataset-to-predict-a-delay?select=Airlines.csv

In [ ]:
import pandas as pd

url="https://raw.githubusercontent.com/degartHub/nocountry-h12-25-equipo27-datascience/refs/heads/main/data/Airlines.csv"
df = pd.read_csv(url)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           539383 non-null  int64 
 1   Airline      539383 non-null  object
 2   Flight       539383 non-null  int64 
 3   AirportFrom  539383 non-null  object
 4   AirportTo    539383 non-null  object
 5   DayOfWeek    539383 non-null  int64 
 6   Time         539383 non-null  int64 
 7   Length       539383 non-null  int64 
 8   Delay        539383 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 37.0+ MB


In [ ]:
df["Time"].agg(["min", "max"])

,Time
min,10
max,1439


La base de datos cuenta con un total de 539.383 registros y un total de 9 columnas, siendo estas:

- <u>**id**</u>= Identifica la fila del registro.

- <u>**Airline**</u>= Aerolínea.

- <u>**Flight**</u>= Número de la aeronave.

- <u>**Airport From**</u>= Aeropuerto de salida.

- <u>**Airport To**</u>= Aeropuerto de destino.

- <u>**DayOfWeek**</u>= Día de la semana (en números).

- <u>**Time**</u>= Hora de salida medida en minutos a partir de la medianoche (rango de [10,1439], lo que podría ser el equivalente a un día).

- <u>**Lenght**</u>= Duración del vuelo en minutos.

- <u>**Delay**</u>= Con retraso (1), sin retraso (0).

In [ ]:
df.sample(n=5)

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
422394,422395,OO,6629,PIA,ORD,6,660,49,0
87726,87727,AA,1256,MIA,LGA,1,465,170,1
119251,119252,CO,86,EWR,CLE,2,1215,98,0
55077,55078,DL,2610,DFW,ATL,6,420,130,1
394907,394908,AS,152,OTZ,ANC,4,895,85,1


Las colummnas a eliminar serán:
- ID: Es un identificador para la tabla en sí
- Flight: Identifica el número de avión, no es relevante.

In [ ]:
df = df.drop(columns=["id", "Flight"])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Airline      539383 non-null  object
 1   AirportFrom  539383 non-null  object
 2   AirportTo    539383 non-null  object
 3   DayOfWeek    539383 non-null  int64 
 4   Time         539383 non-null  int64 
 5   Length       539383 non-null  int64 
 6   Delay        539383 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 28.8+ MB


## Feature Architect (FA)

Sección para las tareas de Feature Architect

Encargado: Eduardo Ayala

### Ingeniería de Atributos

**Acerca de los datos faltantes**

Si observamos los valores mínimos y máximos de la columna `Time` se ve que están en el rango (10, 1439), que corresponde aproximadamente a los minutos que tiene un día completo, y, aparte, hay una columna 'Lenght' con el tiempo de vuelo. Como son ~540 mil vuelos no se puede asumir que son todos del mismo día, es algo más parecido a los vuelos de 1 mes en USA. Así que se crearán las fechas de partida de manera sintética para 1 mes.

In [ ]:
import numpy as np
import pandas as pd

# ---------------------------------------------------------------------
#  CONFIGURACIÓN INICIAL
# ---------------------------------------------------------------------
# Reproducibilidad
np.random.seed(42)

# Definir rango de fechas (solo diciembre 2018)
start_date = pd.to_datetime('2018-12-01')
end_date = pd.to_datetime('2018-12-31')

# ---------------------------------------------------------------------
#  CREAR FECHAS ALEATORIAS (SOLO DÍA, SIN HORAS)
# ---------------------------------------------------------------------
random_days = np.random.randint(
    0,
    (end_date - start_date).days + 1,
    size=len(df)
)

# Columna FlightDate normalizada (00:00:00)
df['FlightDate'] = (
    start_date + pd.to_timedelta(random_days, unit='D')
).normalize()

# ---------------------------------------------------------------------
#  CREAR FECHA-HORA DE SALIDA Y LLEGADA
# ---------------------------------------------------------------------
# Time   = minutos desde las 00:00
# Length = duración del vuelo en minutos

# Fecha y hora de salida
df['DepartureDateTime'] = (
    df['FlightDate']
    + pd.to_timedelta(df['Time'], unit='m')
)

# Fecha y hora de llegada
df['ArrivalDateTime'] = (
    df['DepartureDateTime']
    + pd.to_timedelta(df['Length'], unit='m')
)

# ---------------------------------------------------------------------
#  VERIFICACIÓN RÁPIDA
# ---------------------------------------------------------------------
print(df[['FlightDate', 'DepartureDateTime', 'ArrivalDateTime']].head())
print("\nDías únicos:", df['FlightDate'].dt.date.nunique())
print("Horas en FlightDate:", df['FlightDate'].dt.hour.unique())

  FlightDate   DepartureDateTime     ArrivalDateTime
0 2018-12-07 2018-12-07 00:15:00 2018-12-07 03:40:00
1 2018-12-20 2018-12-20 00:15:00 2018-12-20 03:57:00
2 2018-12-29 2018-12-29 00:20:00 2018-12-29 03:05:00
3 2018-12-15 2018-12-15 00:20:00 2018-12-15 03:35:00
4 2018-12-11 2018-12-11 00:30:00 2018-12-11 03:52:00

Días únicos: 31
Horas en FlightDate: [0]


In [ ]:
# ---------------------------------------------------------------------
#  PARTE 2: CREAR VARIABLES TEMPORALES
# ---------------------------------------------------------------------
# A partir de columnas datetime ya existentes:
# - FlightDate
# - DepartureDateTime
# - ArrivalDateTime

# ---------------------------------------------------------------------
#  VARIABLES DE SALIDA
# ---------------------------------------------------------------------
df['DepartureHour'] = df['DepartureDateTime'].dt.hour
df['DepartureDayOfWeek'] = df['DepartureDateTime'].dt.dayofweek

# ---------------------------------------------------------------------
#  VARIABLES DE LLEGADA
# ---------------------------------------------------------------------
df['ArrivalHour'] = df['ArrivalDateTime'].dt.hour
df['ArrivalDayOfWeek'] = df['ArrivalDateTime'].dt.dayofweek

# ---------------------------------------------------------------------
#  VERIFICACIÓN RÁPIDA
# ---------------------------------------------------------------------
print(
    df[
        [
            'DepartureDateTime',
            'DepartureHour',
            'DepartureDayOfWeek',
            'ArrivalDateTime',
            'ArrivalHour',
            'ArrivalDayOfWeek'
        ]
    ].head()
)

    DepartureDateTime  DepartureHour  DepartureDayOfWeek     ArrivalDateTime  \
0 2018-12-07 00:15:00              0                   4 2018-12-07 03:40:00   
1 2018-12-20 00:15:00              0                   3 2018-12-20 03:57:00   
2 2018-12-29 00:20:00              0                   5 2018-12-29 03:05:00   
3 2018-12-15 00:20:00              0                   5 2018-12-15 03:35:00   
4 2018-12-11 00:30:00              0                   1 2018-12-11 03:52:00   

   ArrivalHour  ArrivalDayOfWeek  
0            3                 4  
1            3                 3  
2            3                 5  
3            3                 5  
4            3                 1  


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# ---------------------------------------------------------------------
#  PARTE 3: ONE-HOT ENCODING DE VARIABLES CATEGÓRICAS
# ---------------------------------------------------------------------

# Variables categóricas a codificar
categorical_features = [
    'Airline',
    'AirportFrom',
    'AirportTo'
]

# Inicializar encoder
# handle_unknown='ignore' permite usar el modelo en producción
ohe = OneHotEncoder(
    sparse_output=False,
    handle_unknown='ignore'
)

# Aplicar One-Hot-Encoding
X_cat = ohe.fit_transform(df[categorical_features])

# Convertir a DataFrame para mejor legibilidad
X_cat = pd.DataFrame(
    X_cat,
    columns=ohe.get_feature_names_out(categorical_features),
    index=df.index
)

# ---------------------------------------------------------------------
#  VERIFICACIÓN RÁPIDA
# ---------------------------------------------------------------------
print("Shape variables categóricas codificadas:", X_cat.shape)
print(X_cat.head())

Shape variables categóricas codificadas: (539383, 604)
   Airline_9E  Airline_AA  Airline_AS  Airline_B6  Airline_CO  Airline_DL  \
0         0.0         0.0         0.0         0.0         1.0         0.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         1.0         0.0         0.0         0.0         0.0   
3         0.0         1.0         0.0         0.0         0.0         0.0   
4         0.0         0.0         1.0         0.0         0.0         0.0   

   Airline_EV  Airline_F9  Airline_FL  Airline_HA  ...  AirportTo_TXK  \
0         0.0         0.0         0.0         0.0  ...            0.0   
1         0.0         0.0         0.0         0.0  ...            0.0   
2         0.0         0.0         0.0         0.0  ...            0.0   
3         0.0         0.0         0.0         0.0  ...            0.0   
4         0.0         0.0         0.0         0.0  ...            0.0   

   AirportTo_TYR  AirportTo_TYS  AirportTo_

**Uso de escalado según el modelo**

El escalado de variables numéricas no siempre es necesario y su aplicación depende del tipo de modelo de machine learning que se esté utilizando. En el caso de la **Regresión Logística**, el escalado **sí es necesario**, ya que este modelo es sensible a la magnitud de las variables numéricas. Al basarse en combinaciones lineales y procesos de optimización por gradiente, el escalado permite un entrenamiento más estable y una correcta asignación de pesos a cada variable. Por otro lado, **Random Forest** **no requiere escalado**, ya que al estar basado en árboles de decisión realiza divisiones por umbrales y no utiliza distancias ni gradientes, por lo que la escala de las variables no influye en su desempeño.

**Aplicación en el código**

En el código se crearon dos conjuntos de datos distintos para respetar los
requisitos de cada modelo. Para Regresión Logística, las variables numéricas
(`Length`, `DepartureHour`, `ArrivalHour`) fueron escaladas utilizando
`StandardScaler`. Para Random Forest, las mismas variables se utilizaron en su
escala original. Ambos conjuntos de datos comparten las variables categóricas
transformadas mediante One-Hot Encoding, y los objetos de transformación fueron
guardados para su uso posterior en producción.

In [ ]:
from sklearn.preprocessing import StandardScaler

# ---------------------------------------------------------------------
#  PARTE 4: ESCALADO Y DATASETS POR MODELO
# ---------------------------------------------------------------------

# Variable objetivo
y = df['Delay']

# Variables numéricas seleccionadas
numeric_features = [
    'Length',
    'DepartureHour',
    'ArrivalHour'
]

X_num = df[numeric_features]

# ---------------------------------------------------------------------
#  DATASET PARA RANDOM FOREST (SIN ESCALADO)
# ---------------------------------------------------------------------
X_rf = pd.concat([X_num, X_cat], axis=1)

print("Shape X_rf:", X_rf.shape)

# ---------------------------------------------------------------------
#  DATASET PARA REGRESIÓN LOGÍSTICA (CON ESCALADO)
# ---------------------------------------------------------------------
scaler = StandardScaler()

X_num_scaled = scaler.fit_transform(X_num)

X_num_scaled = pd.DataFrame(
    X_num_scaled,
    columns=numeric_features,
    index=df.index
)

X_logreg = pd.concat([X_num_scaled, X_cat], axis=1)

print("Shape X_logreg:", X_logreg.shape)

In [ ]:
import joblib

# ---------------------------------------------------------------------
#  PARTE 5: GUARDAR OBJETOS DE TRANSFORMACIÓN
# ---------------------------------------------------------------------
# Estos objetos se reutilizan en inferencia para garantizar
# consistencia entre entrenamiento y producción

joblib.dump(scaler, 'scaler_logreg.pkl')
joblib.dump(ohe, 'onehot_encoder.pkl')

print("Objetos de transformación guardados correctamente")

**Reducción de memoria en el dataset (opcional)**

Para poder entrenar los modelos con todo el dataset sin saturar la RAM, realizamos los siguientes cambios:

1. Cambio de tipos de datos:
   - Variables numéricas (Length, DepartureHour, ArrivalHour) convertidas de float64 a float32
     - Esto reduce a la mitad el tamaño de cada celda.
   - Variables categóricas One-Hot (0.0 / 1.0) convertidas a uint8
     - Cada celda ocupa solo 1 byte en lugar de 8 bytes.

2. Uso de sparse matrices:
   - One-Hot Encoding genera muchas columnas con ceros.
   - Convertir el DataFrame a sparse.csr_matrix evita almacenar los ceros.
   - Esto ahorra memoria y permite entrenar Regresión Logística sin sample.

3. Escalado eficiente:
   - Para Regresión Logística, las columnas numéricas se escalaron con StandardScaler(with_mean=False)
   - with_mean=False es obligatorio para sparse matrices, evitando densificar el dataset.

Con estos cambios, el dataset pasó de varios GB a menos de 1 GB, permitiendo entrenar con todo el dataset y manteniendo un rendimiento estable.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import sparse

# ---------------------------------------------------------------------
#  REDUCCIÓN DE MEMORIA
# ---------------------------------------------------------------------

# Variables numéricas
numeric_features = ['Length', 'DepartureHour', 'ArrivalHour']

# Convertir numéricas a float32
X_logreg[numeric_features] = X_logreg[numeric_features].astype('float32')
X_rf[numeric_features] = X_rf[numeric_features].astype('float32')

# Convertir variables categóricas One-Hot a uint8
X_logreg[X_cat.columns] = X_cat.astype('uint8')
X_rf[X_cat.columns] = X_cat.astype('uint8')

print("Tipos de datos convertidos. Memoria reducida.")

# ---------------------------------------------------------------------
#  OPCIONAL: convertir a sparse matrices para ahorrar más memoria
# ---------------------------------------------------------------------
# Para Regresión Logística
X_logreg_sparse = sparse.csr_matrix(X_logreg.values)

# Para Random Forest (opcional, RF puede manejar DataFrame normal)
X_rf_sparse = sparse.csr_matrix(X_rf.values)

print("Datasets convertidos a sparse. Listos para entrenar.")
print("X_logreg_sparse shape:", X_logreg_sparse.shape)
print("X_rf_sparse shape:", X_rf_sparse.shape)

# ---------------------------------------------------------------------
#  ESCALADO DE VARIABLES NUMÉRICAS PARA LOGREG
# ---------------------------------------------------------------------
scaler = StandardScaler(with_mean=False)  # with_mean=False es obligatorio para sparse

X_logreg_scaled = X_logreg_sparse.copy()
X_logreg_scaled[:, :len(numeric_features)] = scaler.fit_transform(X_logreg_sparse[:, :len(numeric_features)])

print("Variables numéricas escaladas. Dataset listo para Regresión Logística.")

## Machine Learning Engineer (MLE)

Sección para las tareas de Machine Learning Engineer

Encargado: Luis Jácome

### Entrenamiento y Evaluación base

## Machine Learning Operations (MLOps)

Sección para las tareas de Machine Learning Operations

Encargado: Nicolás Staffelbach

### Microservicio Python

## Data Analyst (DA)

Sección para las tareas de Data Analyst

Encargado: David Aragón

### Análisis de Datos Exploratorio EDA